In [1]:
# Loading the modules used for the analysis

import numpy as np     # To be able to store our numbers in numpy formats already
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from numpy import random
import pandas as pd # Pandas should facilitate a cleaner data analysis
import seaborn as sns
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import wasserstein_distance
import itertools
from scipy import stats


plt.rcParams['figure.figsize'] = [12, 7] # Size of the graphs in inches

evenly_spaced_interval = np.linspace(0, 1, 19)

colors = [cm.tab20(x) for x in evenly_spaced_interval]

### Pre-curating the data

The data will be imported as a list of dictionaries. We also do some pre-processing to get rid of structures where the functional group detection had failed as well as those for which DFT polarizability is not available

In [2]:
import pickle    # The data was saved as a pickle file to make reading and transferring easier
data = []
with open('pickled_data_rev2.txt', 'rb') as file:
        try:
                while True: # Here go all the operations, element by element
                        currdata = pickle.load(file)
                        if "-c1-" in str(currdata[0][1]): # We are NOT interested in conformers, we only load the first conformer for every data here
                                data.append(currdata)
        except EOFError: # Finished with the reading
            pass

In [3]:
# Creating a list of dictionaries that can be searched for various properties
# The only goal is to make the file more readable
                    
alldata = []

for entry in data:
    alldata.append( { 'molid': entry[0][0], 'confid': entry[0][1] , 'atypes': entry[0][2], 
            'hlgap': entry[0][3][0], 'mbd_alpha': entry[0][4][0], 'size': entry[0][5], 
            'dft_alpha': entry[0][6], 'groups': entry[0][7] } )
    
# An example to filter the dataset to all molecules where no functional groups are detected at all

#list(filter(lambda nogroup: nogroup['groups'] == ['not found'], alldata))

print(len(alldata))

13278


In [4]:
# Removing all elements where zero functional groups have been detected

nogroups = list(filter(lambda nogroup: nogroup['groups'] == ['not found'], alldata))

alldata_withgroups = [item for item in alldata if item not in nogroups]

del nogroups # freeing up memory
alldata = alldata_withgroups
del alldata_withgroups

In [5]:
# Adding a few more numbers to the dictionaries

ii=0
for entry in alldata:
    alphas, atoms, sizes = entry["dft_alpha"], entry["atypes"], entry["size"]
    alldata[ii].update( { 'dft_alpha_avg' : (alphas[0]+alphas[1]+alphas[2])/3, 
                      "nelectrons" : np.float64( np.sum( atoms)), "maxsize" : np.float64(sizes[0])} )
    ii+=1

In [6]:
# Removing cases where the DFT calculation had failed

nopol = list(filter(lambda failed_pol: failed_pol['dft_alpha_avg'] < -90, alldata))

alldata_withpol = [item for item in alldata if item not in nopol]

del nopol # freeing up memory
alldata = alldata_withpol
del alldata_withpol

### Loading the data is done

The points are stored in alldata. An example data point is

In [7]:
for key, value in alldata[100].items():
    print(key, ' : ', value)

molid  :  168
confid  :  Geom-m168-i2-c1-opt
atypes  :  [6 6 6 6 8 1 1 1 1 1 1]
hlgap  :  8.036732
mbd_alpha  :  63.485137
size  :  [5.588716978592045, 1.537705, 0.9684630000000001, 3.104132]
dft_alpha  :  [47.31031, 50.0454, 54.54521, 7.58871, -6.22808, -10.2374]
groups  :  ['secondary alcohol', 'alkyne']
dft_alpha_avg  :  50.63363999999999
nelectrons  :  38.0
maxsize  :  5.588716978592045
